# ECOモデルの実装

## 目標
1. ECOの２D Netモジュールの概要を理解
2. Inception-v2を実装する

1. ECOの3D Netモジュールの概要を理解
2. 3D resnetを実装できるようになる

## Library

In [24]:
import torch
from torch import nn

## 動画データに対するディープラーニング手法
1. C3D<br>
CNNを３次元に拡張したようなもの<br>
2. Two-Stream ConvNets<br>
画像情報に加えてオプティカルフローという物体が移動した奇跡をベクトルで表したものを使う


両者とも時間軸を考慮している

## ECOの2D NETモジュール
ひらすら実装する

In [25]:
class BasicConv(nn.Module):
    '''
    ECOの２D Netモジュールの最初のモジュール
    '''
    
    def __init__(self):
        super(BasicConv, self).__init__()
        
        # s2とreduceの意味はなにか？
        # reduce pointwise畳み込みのこと？今回は次元圧縮になってないけど、次元削減sることが多いから？
        # s2は
        self.conv1_7x7_s2 = nn.Conv2d(
            3, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.conv1_7x7_s2_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv1_relu_7x7 = nn.ReLU(inplace=True)
        self.pool1_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        
        self.conv2_3x3_reduce = nn.Conv2d(
            64, 64, kernel_size=(1,1), stride=(1,1))
        self.conv2_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.conv2_3x3 = nn.Conv2d(
            64, 192, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2_3x3_bn = nn.BatchNorm2d(
            192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2_relu_3x3 = nn.ReLU(inplace=True)
        self.pool2_3x3_s2 = nn.MaxPool2d(
            kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
        
    def forward(self, x):
        out = self.conv1_7x7_s2(x)
        out = self.conv1_7x7_s2_bn(out)
        out = self.conv1_relu_7x7(out)
        out = self.pool1_3x3_s2(out)
        out = self.conv2_3x3_reduce(out)
        out = self.conv2_3x3_reduce_bn(out)
        out = self.conv2_relu_3x3_reduce(out)
        out = self.conv2_3x3(out)
        out = self.conv2_3x3_bn(out)
        out = self.conv2_relu_3x3(out)
        out = self.pool2_3x3_s2(out)
        return out

In [26]:
class InceptionA(nn.Module):
    '''InceptionA'''
    
    def __init__(self):
        super(InceptionA, self).__init__()
        
        self.inception_3a_1x1 = nn.Conv2d(
            192, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3a_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_1x1 = nn.ReLU(inplace=True)
        
        self.inception_3a_3x3_reduce = nn.Conv2d(192, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3a_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_3x3 = nn.Conv2d(
            64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3a_3x3_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_3x3 = nn.ReLU(inplace=True)
        
        self.inception_3a_double_3x3_reduce = nn.Conv2d(192, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3a_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3a_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3a_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3a_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_double_3x3_2 = nn.ReLU(inplace=True)
        
        self.inception_3a_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3a_pool_proj = nn.Conv2d(
            192, 32, kernel_size=(1,1), stride=(1,1))
        self.inception_3a_pool_proj_bn = nn.BatchNorm2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3a_relu_pool_proj = nn.ReLU(inplace=True)
        
        def forward(self, x):
            out1= self.inception_3a_1x1(x)
            out1= self.inception_3a_1x1_bn(out1)
            out1= self.inception_3a_relu_1x1(out1)
            
            out2 = self.inception_3a_3x3_reduce(x)
            out2 = self.inception_3a_3x3_reduce_bn(out2)
            out2 = self.inception_3a_relu_3x3_reduce(out2)
            out2 = self.inception_3a_3x3(out2)
            out2 = self.inception_3a_3x3_bn(out2)
            out2 = self.inception_3a_relu_3x3(out2)
            
            out3 = self.inception_3a_double_3x3_reduce(x)
            out3 = self.inception_3a_double_3x3_reduce_bn(out3)
            out3 = self.inception_3a_relu_double_3x3_reduce(out3)
            out3 = self.inception_3a_double_3x3_1(out3)
            out3 = self.inception_3a_double_3x3_1_bn(out3)
            out3 = self.inception_3a_relu_double_3x3_1(out3)
            out3 = self.inception_3a_double_3x3_2(out3)
            out3 = self.inception_3a_double_3x3_2_bn(out3)
            out3 = self.inception_3a_relu_double_3x3_2(out3)
            
            out4 = self.inception_3a_pool(x)
            out4 = self.inception_3a_pool_proj(out4)
            out4 = self.inception_3a_pool_proj_bn(out4)
            out4 = self.inception_3a_relu_pool_proj(out4)
            
            outputs = [out1, out2, out3, out4]
            
            return torch.cat(outputs, 1)  # channelの軸で結合
    
            

In [27]:
# InceptionAとほぼ同じだが、channelの数が違う部分がある

class InceptionB(nn.Module):
    '''InceptionB'''
    
    def __init__(self):
        super(InceptionB, self).__init__()
        
        self.inception_3b_1x1 = nn.Conv2d(
            256, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3b_1x1_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_1x1 = nn.ReLU(inplace=True)
        
        self.inception_3b_3x3_reduce = nn.Conv2d(256, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3b_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_3x3 = nn.Conv2d(
            64, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3b_3x3_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_3x3 = nn.ReLU(inplace=True)
        
        self.inception_3b_double_3x3_reduce = nn.Conv2d(256, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3b_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_1 = nn.Conv2d(
            64, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3b_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_1 = nn.ReLU(inplace=True)
        self.inception_3b_double_3x3_2 = nn.Conv2d(
            96, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3b_double_3x3_2_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_double_3x3_2 = nn.ReLU(inplace=True)
        
        self.inception_3b_pool = nn.AvgPool2d(
            kernel_size=3, stride=1, padding=1)
        self.inception_3b_pool_proj = nn.Conv2d(
            256, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3b_pool_proj_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3b_relu_pool_proj = nn.ReLU(inplace=True)
        
        def forward(self, x):
            out1= self.inception_3b_1x1(x)
            out1= self.inception_3b_1x1_bn(out1)
            out1= self.inception_3b_relu_1x1(out1)
            
            out2 = self.inception_3b_3x3_reduce(x)
            out2 = self.inception_3b_3x3_reduce_bn(out2)
            out2 = self.inception_3b_relu_3x3_reduce(out2)
            out2 = self.inception_3b_3x3(out2)
            out2 = self.inception_3b_3x3_bn(out2)
            out2 = self.inception_3b_relu_3x3(out2)
            
            out3 = self.inception_3b_double_3x3_reduce(x)
            out3 = self.inception_3b_double_3x3_reduce_bn(out3)
            out3 = self.inception_3b_relu_double_3x3_reduce(out3)
            out3 = self.inception_3b_double_3x3_1(out3)
            out3 = self.inception_3b_double_3x3_1_bn(out3)
            out3 = self.inception_3b_relu_double_3x3_1(out3)
            out3 = self.inception_3b_double_3x3_2(out3)
            out3 = self.inception_3b_double_3x3_2_bn(out3)
            out3 = self.inception_3b_relu_double_3x3_2(out3)
            
            out4 = self.inception_3b_pool(x)
            out4 = self.inception_3b_pool_proj(out4)
            out4 = self.inception_3b_pool_proj_bn(out4)
            out4 = self.inception_3b_relu_pool_proj(out4)
            
            outputs = [out1, out2, out3, out4]
            
            return torch.cat(outputs, 1)  # channelの軸で結合

In [33]:
# 分岐なし

class InceptionC(nn.Module):
    '''InceptionC'''
    
    def __init__(self):
        super(InceptionC, self).__init__()
        
        self.inception_3c_double_3x3_reduce = nn.Conv2d(320, 64, kernel_size=(1,1), stride=(1,1))
        self.inception_3c_double_3x3_reduce_bn = nn.BatchNorm2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_reduce = nn.ReLU(inplace=True)
        self.inception_3c_double_3x3_1 = nn.Conv2d(64, 96, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.inception_3c_double_3x3_1_bn = nn.BatchNorm2d(
            96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.inception_3c_relu_double_3x3_1 = nn.ReLU(inplace=True)
        
    def forward(self, x):
        out = self.inception_3c_double_3x3_reduce(x)
        out = self.inception_3c_double_3x3_reduce_bn(out)
        out = self.inception_3c_relu_double_3x3_reduce(out)
        out = self.inception_3c_double_3x3_1(out)
        out = self.inception_3c_double_3x3_1_bn(out)
        out = self.inception_3c_relu_double_3x3_1(out)
        
        return out
          
        

In [34]:
class ECO_2D(nn.Module):
    def __init__(self):
        super(ECO_2D, self).__init__()
        
        # BasicConvモジュール
        self.basic_conv = BasicConv()
        
        # Inceptionモジュール
        self.inception_a = InceptionA()
        self.inception_b = InceptionB()
        self.inception_c = InceptionC()
        
    def forward(self, x):
        # Inceptionの中では分岐があるけれど、ECO全体の流れとしては直列
        '''
        入力のサイズ torch.Size([batch_num, 3, 224, 224])
        '''
        out = self.basic_conv(x)
        out = self.inception_a(out)
        out = self.inception_b(out)
        out = self.inception_c(out)
        
        return out

## 動作確認

In [35]:
# モデルの用意
net = ECO_2D()
net.train()

ECO_2D(
  (basic_conv): BasicConv(
    (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_relu_7x7): ReLU(inplace=True)
    (pool1_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3_reduce): ReLU(inplace=True)
    (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_relu_3x3): ReLU(inplace=True)
    (pool2_3x3_s2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (inception_a): InceptionA(
    (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), 

In [ ]:
# 1. tensorboardXの保存クラスを呼び出す
from tensorboardX import SummaryWriter

# 2. フォルダ「tbX」に保存させるwriterを用意
# フォルダ「tbX」がなければ自動作成
writer = SummaryWriter('./tbX/')

# 3. ネットワークに流すダミーデータを作成
batch_size = 1
dummpy_img = torch.rand(batch_size, 3, 224, 224)

# 4. netに対してダミーデータを流したときのgraphをwriterに保存
writer.add_graph(net, (dummpy_img,))
writer.close()

# 5. コマンドプロンプトでフォルダtbXがあるフォルダまで移動して、以下のコマンドを実行
# tensorboard --logdir="./tbX/"
# その後、http://localhost:6006 にアクセス

## ECOの3D Netモジュール

In [36]:
# 最初のテンソルの入れ替えはモジュール内に実装せず、forward関数内に実装する

class Resnet_3D_3(nn.Module):
    '''
    Resnet_3D_3
    '''
    
    def __init__(self):
        super(Resnet_3D_3, self).__init__()
        
        self.res3a_2 = nn.Conv3d(
            96, 128, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        self.res3a_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3a_relu = nn.ReLU(inplace=True)
        
        self.res3b_1 = nn.Conv3d(
            128, 128, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        self.res3b_1_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_1_relu = nn.ReLU(inplace=True)
        self.res3b_2 = nn.Conv3d(
            128, 128, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        
        self.res3b_bn = nn.BatchNorm3d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res3b_relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        residual = self.res3a_2(x)  # 後で加算させるのでoutでなくresidualとして保管しておく
        out = self.res3a_bn(residual)
        out = self.res3a_relu(out)
        
        out = self.res3b_1(out)
        out = self.res3b_1_bn(out)
        out = self.res3b_1_relu(out)
        out = self.res3b_2(out)
        
        out += residual
        
        out = self.res3b_bn(out)
        out = self.res3b_relu(out)
        
        return out

In [37]:
class Resnet_3D_4(nn.Module):
    '''
    Resnet_3D_4
    '''
    
    def __init__(self):
        super(Resnet_3D_4, self).__init__()
        
        self.res4a_1 = nn.Conv3d(
            128, 256, kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1))
        self.res4a_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_1_relu = nn.ReLU(inplace=True)
        self.res4a_2 =  nn.Conv3d(
            256, 256, kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1))
        
        self.res4a_down = nn.Conv3d(
            128, 256, kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1))
        
        self.res4a_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4a_relu = nn.ReLU(inplace=True)
        
        self.res4b_1 = nn.Conv3d(
            256, 256, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        self.res4b_1_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_1_relu = nn.ReLU(inplace=True)
        self.res4b_2 = nn.Conv3d(
            256, 256, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        
        self.res4b_bn = nn.BatchNorm3d(
            256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res4b_relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        residual = self.res4a_down(x)  # 後で加算させるのでoutでなくresidualとして保管しておく
        
        out = self.res4a_1(x)
        out = self.res4a_1_bn(out)
        out = self.res4a_1_relu(out)
        
        out += self.res4a_2(out)
        
        out += residual
        
        residual2 = out
        
        out = self.res4a_bn(out)
        out = self.res4a_relu(out)
        
        out = self.res4b_1(out)
        out = self.res4b_1_bn(out)
        out = self.res4b_1_relu(out)
        
        out = self.res4b_2(out)
        
        out += residual2
        
        out = self.res4b_bn(out)
        out = self.res4b_relu(out)
        
        return out

In [38]:
class Resnet_3D_5(nn.Module):
    '''
    Resnet_3D_5
    '''
    
    def __init__(self):
        super(Resnet_3D_5, self).__init__()
        
        self.res5a_1 = nn.Conv3d(
            256, 512, kernel_size=(3,3,3), stride=(2,2,2), padding=(1,1,1))
        self.res5a_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_1_relu = nn.ReLU(inplace=True)
        self.res5a_2 = nn.Conv3d(
            512, 512, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        
        self.res5a_down = nn.Conv3d(
            256, 512, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        
        self.res5a_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5a_relu = nn.ReLU(inplace=True)
        
        self.res5b_1 = nn.Conv3d(
            512, 512, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        self.res5b_1_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_1_relu = nn.ReLU(inplace=True)
        self.res5b_2 = nn.Conv3d(
            512, 512, kernel_size=(3,3,3), stride=(1,1,1), padding=(1,1,1))
        
        self.res5b_bn = nn.BatchNorm3d(
            512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.res5b_relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        residual = self.res5a_down(x)  # 後で加算させるのでoutでなくresidualとして保管しておく
        
        out = self.res5a_1(x)
        out = self.res5a_1_bn(out)
        out = self.res5a_1_relu(out)
        
        out = self.res5a_2(out)
        
        out += residual
        
        residual2 = out
        
        out = self.res5a_bn(out)
        out = self.res5a_relu(out)
        
        out = self.res5b_1(out)
        out = self.res5b_1_bn(out)
        out = self.res5b_1_relu(out)
        
        out = self.res5b_2(out)
        
        out += residual2
        
        out = res5b_bn(out)
        out = res5b_relu(out)
        
        return out
        

In [40]:
class ECO_3D(nn.Module):
    def __init__(self):
        super(ECO_3D, self).__init__()
        
        # 3D_Resnetモジュール
        # 上で実装した３つのresnetモジュールをまとめる
        self.res_3d_3 = Resnet_3D_3()
        self.res_3d_4 = Resnet_3D_4()
        self.res_3d_5 = Resnet_3D_5()
        
        # global average pooling
        # 全結合層にしたほうが複雑になり表現力も増すが、パラメータが多くなり過学習しやすくなる
        # 全結合層の代わりに使用するAveragePoolingをglobal average poolingという
        self.global_pool = nn.AvgPool3d(
            kernel_size=(4, 7, 7), stride=1, padding=0)
        
    def forward(self, x):
        '''
        入力サイズのtorch.Size([batch_num, frames, 96, 28, 28])
        '''
        # （batch_num, 時間(frames), チャネル, 高さ, 幅）→　（batch_num, チャネル, 時間(frames), 高さ, 幅）
        out = torch.transpose(x, 1, 2)  # テンソルの順番の入れ替え　
        out = self.res_3d_3(out)
        out = self.res_3d_4(out)
        out = self.res_3d_5(out)
        out = self.global_pool(out)
        
        # テンソルサイズを変更
        # torch.Size([batch_num, 512, 1, 1]) →　torch.Size([batch_num, 512])
        out = out.view(out.size()[0], out.size()[1])
        # viewを使うときは入力テンソルがメモリ上でも要素順に並んでいる必要がある
        
        return out
        

## 動作確認

In [41]:
# モデルの用意
net = ECO_3D()
net.train()

ECO_3D(
  (res_3d_3): Resnet_3D_3(
    (res3a_2): Conv3d(96, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3a_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3a_relu): ReLU(inplace=True)
    (res3b_1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_1_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_1_relu): ReLU(inplace=True)
    (res3b_2): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (res3b_bn): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res3b_relu): ReLU(inplace=True)
  )
  (res_3d_4): Resnet_3D_4(
    (res4a_1): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (res4a_1_bn): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (res4a_1_relu): ReLU(inplace=True)
    (res4a_2): Conv3d(2

In [ ]:
# 1. tensorboardXの保存クラスを呼び出す
from tensorboardX import SummaryWriter

# 2. フォルダ「tbX」に保存させるwriterを用意
# フォルダ「tbX」がなければ自動作成
writer = SummaryWriter('./tbX/')

# 3. ネットワークに流すダミーデータを作成
batch_size = 1
dummpy_img = torch.rand(batch_size, 16, 96, 28, 28)

# 4. netに対してダミーデータを流したときのgraphをwriterに保存
writer.add_graph(net, (dummpy_img,))
writer.close()

# 5. コマンドプロンプトでフォルダtbXがあるフォルダまで移動して、以下のコマンドを実行
# tensorboard --logdir="./tbX/"
# その後、http://localhost:6006 にアクセス